<a href="https://colab.research.google.com/github/leonardoLavagna/Kaggle-Competition/blob/main/KaggleCompetitionFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [ ]:
# Standard imports
import time
import os
import copy
import tqdm.notebook as tq
from csv import writer

import seaborn as sn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from PIL import Image

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder

In [ ]:
# Configuration
IMAGE_SIZE = 224
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
PATH = '/content'
BATCHES = 32
NUM_WORKERS = 2
EPOCHS = 20
RANDOM_STATE = 1234
N_CLASSES = 7
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Remamber to import the images on Google Colab. Execute the following command
# !git clone https://github.com/leonardoLavagna/Kaggle-Competition.git
# After that you need to adjust the directory tree in Colab.
# 1) Put the images folder inside the contents directory
# 2) Put the test folder in the contents directory
# 3) Create a nested folder with the previoucly moved test folder:

Cloning into 'Kaggle-Competition'...
remote: Enumerating objects: 5251, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 5251 (delta 11), reused 13 (delta 4), pack-reused 5218
Receiving objects: 100% (5251/5251), 129.75 MiB | 17.03 MiB/s, done.
Resolving deltas: 100% (13/13), done.


# Load the data

In [ ]:
diz ={"battleships":0,"coast-guard":1,"containerships":2,"cruise-ships":3,"drilling-rigs":4,"motor-yachts":5,"submarines":6}

def createCsvTraning():
  with open('TrainingDataset.csv', 'wt') as f_object:
    writer_object = writer(f_object)
    row = ["file_name","category_name","category_id","path"]
    writer_object.writerow(row)
    path = "/content/images/train/"
    for dir in os.listdir(path):
      if not dir.startswith("."):
        for file in os.listdir(path+dir):
            if not file.startswith("."):
                row = [file,dir,diz[dir],path+dir+"/"+file]
                writer_object.writerow(row)

createCsvTraning()

In [ ]:
df = pd.read_csv(PATH+"/TrainingDataset.csv") #TrainingDataset.csv

In [ ]:
# check
#df.head()

In [ ]:
df_train, df_val = train_test_split(df, stratify=df.category_id, test_size=0.2, random_state=RANDOM_STATE)
df_train.reset_index(inplace=True)
df_val.reset_index(inplace=True)

In [ ]:
input_size = (IMAGE_SIZE, IMAGE_SIZE)

In [ ]:
class ShipDataset:
    def __init__(self, root, df, transform):
        self.root = root
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        cat = self.df.category_name[index]
        cat_id = self.df.category_id[index]
        file_name = self.df.file_name[index]
        img_path = os.path.join(self.root, cat, file_name)
  
        with open(img_path, "rb") as fp:
          img = Image.open(fp).convert("RGB")

        img = self.transform(img)
        return img, cat_id

In [ ]:
train_transform = transforms.Compose([transforms.RandomResizedCrop(input_size),
                            transforms.RandomHorizontalFlip(),
                            transforms.ToTensor(),
                            transforms.Normalize(MEAN, STD)
                            ])
# Define the ShipDataset, it will return a tuple: (image, label)
train_dataset = ShipDataset(PATH+"/images/train/", df_train, transform=train_transform)
print("len train_dataset", len(train_dataset))

len train_dataset 1602


In [ ]:
val_transform = transforms.Compose([transforms.Resize(input_size),
                            transforms.ToTensor(),
                            transforms.Normalize(MEAN, STD)
                            ])

val_dataset = ShipDataset(PATH+"/images/train", df_val, transform=val_transform)
print("len val_dataset", len(val_dataset))

len val_dataset 401


In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                            batch_size=BATCHES,
                            num_workers=NUM_WORKERS,
                            shuffle=True,   
                            drop_last=True   
                            )

val_loader = torch.utils.data.DataLoader(val_dataset,
                            batch_size=BATCHES,
                            num_workers=NUM_WORKERS,
                            shuffle=False   
                            )

In [ ]:
'''model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 7)'''
model = torchvision.models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, out_features=N_CLASSES)

nn.init.xavier_normal_(model.fc.weight)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Parameter containing:
tensor([[ 0.0338, -0.0078,  0.0500,  ..., -0.0106,  0.0095, -0.0018],
        [ 0.0181,  0.0084, -0.0181,  ...,  0.0259, -0.0398,  0.0164],
        [-0.0263, -0.0154, -0.0042,  ..., -0.0137,  0.0107, -0.0109],
        ...,
        [ 0.0141, -0.0257,  0.0197,  ..., -0.0715,  0.0237,  0.0075],
        [ 0.0233,  0.0051,  0.0562,  ..., -0.0545, -0.0050,  0.0069],
        [-0.0512,  0.0368,  0.0265,  ..., -0.0075,  0.0611,  0.0169]],
       requires_grad=True)

In [ ]:
model = model.to(DEVICE)
#optimizer = optim.Adam(model.parameters(), lr = 5e-4)
optimizer = torch.optim.SGD(model.parameters(), lr=5e-4, momentum=0.9)
scheluder = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
#scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
cost = torch.nn.CrossEntropyLoss()
# warm-up training
# train(model,cost,optimizer,3)
# torch.save(model.state_dict(), PATH)
# model.load_state_dict(torch.load(PATH))

In [ ]:
def train(model,cost, optimizer_ft,epochs):
    n = len(train_loader)
    for epoch in range(epochs):
        # Train mode
        model.train()
        for iter, (image_batch, label_batch) in enumerate(tq.tqdm(train_loader)):
            torch.cuda.empty_cache()
            # LR updates
            # Stop to accumulate gradients
            optimizer_ft.zero_grad()

            image_batch = image_batch.to(DEVICE)
            label_batch = label_batch.to(DEVICE)
            # compute the logits and squeeze the last dimension
            logits = model(image_batch).squeeze()
            # Compute the mini-batch cost
            cost_tensor = cost(logits, label_batch)
            # sometimes print the loss
            if iter % int(0.25*n) == 0:
                print(f"Epoch {epoch}, iter {iter}/{n} Train cost {cost_tensor.item():.6f}")
            # Gradient computation!
            cost_tensor.backward()
            # Optimizer step!
            optimizer_ft.step()
        scheduler.step()
        # Eval the model after every epoch
        meanf1score, y_true, y_pred = eval(model, val_loader)
        torch.save(model.state_dict(), f"epoch_{epoch}_{meanf1score:.4f}.pth")
        print(f"Val meanF1score: {meanf1score*100:.4f}")


def eval(model, loader):
    y_pred = []
    y_true = []
    # eval mode
    model.eval()
    for images, labels in loader:
        images = images.to(DEVICE)
        with torch.no_grad():
            logits = model(images)
        y_pred += list(torch.argmax(logits,1).cpu().numpy())
        y_true += list(labels.cpu().numpy())

    meanf1score = f1_score(y_true, y_pred, average="macro")

    return meanf1score, y_true, y_pred

In [ ]:
# regular training
train(model,cost,optimizer,EPOCHS)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 0, iter 0/100 Train cost 0.325600
Epoch 0, iter 25/100 Train cost 0.328486
Epoch 0, iter 50/100 Train cost 0.231461
Epoch 0, iter 75/100 Train cost 0.572272
Val meanF1score: 94.2708


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1, iter 0/100 Train cost 0.241211
Epoch 1, iter 25/100 Train cost 0.300110
Epoch 1, iter 50/100 Train cost 0.211289
Epoch 1, iter 75/100 Train cost 0.355298
Val meanF1score: 92.3291


  0%|          | 0/100 [00:00<?, ?it/s]

...

Epoch 17, iter 0/100 Train cost 0.218128
Epoch 17, iter 25/100 Train cost 0.095533
Epoch 17, iter 50/100 Train cost 0.329600
Epoch 17, iter 75/100 Train cost 0.025058
Val meanF1score: 95.9563


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 18, iter 0/100 Train cost 0.162757
Epoch 18, iter 25/100 Train cost 0.029761
Epoch 18, iter 50/100 Train cost 0.172826
Epoch 18, iter 75/100 Train cost 0.056497
Val meanF1score: 94.0986


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 19, iter 0/100 Train cost 0.047458
Epoch 19, iter 25/100 Train cost 0.148328
Epoch 19, iter 50/100 Train cost 0.063143
Epoch 19, iter 75/100 Train cost 0.413638
Val meanF1score: 94.7326


# Testing

In [ ]:
test_dataset = ImageFolder("/content/test", transform=val_transform)

In [ ]:
test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=BATCHES,
                                          num_workers=NUM_WORKERS,
                                          shuffle=False)

In [ ]:
# check
test_dataset.samples[:3]

[('/content/test/test/000c110b.jpg', 0),
 ('/content/test/test/00268327.jpg', 0),
 ('/content/test/test/008fcbc9.jpg', 0)]

In [ ]:
y_pred = []
# eval mode
model.eval()
for images, labels in test_loader:
    images = images.to(DEVICE)
    with torch.no_grad():
        logits = model(images)
    y_pred += list(torch.argmax(logits,1).cpu().numpy())

# Submission file

In [ ]:
df_submission = pd.DataFrame()
df_submission["file_name"] = [os.path.basename(p[0]) for p in test_dataset.samples]
df_submission["category_id"] = y_pred
df_submission.to_csv("submission.csv", index=False)